## Environment Setup

In [ ]:
! pip install langchain_community langchain_mistralai langchainhub langchain tiktoken langchain_pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.10.10
    Uninstalling aiohttp-3.10.10:
      Successfully uninstalled aiohttp-3.10.10
  Attempting uninstall: langchain-core
  

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '<your key goes here>'
os.environ['MISTRAL_API_KEY'] = '<your key goes here>'
os.environ['PINECONE_API_KEY'] = '<your key goes here>'



In [ ]:
questions = [ "What does Moby Dick say about humanity's struggle against nature?",
              "How does Moby Dick explore the theme of obsession through Ahab's quest?",
              "How is Captain Ahab portrayed as both a hero and a villain in Moby Dick?",
              # "What motivates Ishmael to join the Pequod, and how does he change throughout the novel?",
              # "What does the white whale symbolize in Moby Dick, and how does it relate to Ahab's obsession?",
              # "How does the novel Moby Dick use the ocean as a symbol of the unknown?",
              # "How does Melville’s narrative style in Moby Dick contribute to the sense of adventure and mystery?",
              # "How does Ishmael’s perspective shape the reader’s understanding of the story in Moby Dick?",
              # "How does Moby Dick reflect 19th-century views on fate and destiny?",
              # "What philosophical questions does Melville raise about human existence and purpose in Moby Dick?",
              # "How does Melville use imagery to depict the sea as both beautiful and terrifying in Moby Dick?",
              # "How does Moby Dick describe the vastness and danger of the open sea?",
              # "What moral dilemmas do the crew members face in Moby Dick?",
              # "How does Moby Dick present Ahab's pursuit of revenge as both justified and self-destructive?"
              ]

question = questions[0]

## Indexing

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
# there can be mulitple urls
loader = WebBaseLoader("https://www.gutenberg.org/cache/epub/2701/pg2701.txt")
books = loader.load()

### Splitting the text

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(books)
len(splits)

1412

### vectorize chnks

In [ ]:
from langchain_mistralai import MistralAIEmbeddings
from langchain_community.vectorstores import Pinecone

vectorstore = Pinecone.from_documents(
                                            index_name = 'book-passages',
                                            documents = splits,
                                            embedding = MistralAIEmbeddings())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_mistralai/embeddings.py:169: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


## Retrieval

### Init Retriever

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

### Drill-down Prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """Generate five unique variations of the given user question to improve document retrieval from a vector database.
The goal is to create diverse perspectives on the question, helping to mitigate limitations of distance-based similarity search.
Separate each alternative question with a newline. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_mistralai import ChatMistralAI

generate_queries = (
    prompt_perspectives
    | ChatMistralAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

### Output original questions and their rephrased questions

In [ ]:
for question in questions:
    print("-------------")
    print(question)
    rephrased_questions = generate_queries.invoke({"question":question})
    for rephrased_question in rephrased_questions:
        print("\t"+rephrased_question)

-------------
What does Moby Dick say about humanity's struggle against nature?
	1. In what ways does Herman Melville's "Moby Dick" explore the theme of human conflict with nature?
	2. How does the novel "Moby Dick" portray the struggle between mankind and natural forces?
	3. Examine the depiction of humanity's battle against nature as presented in "Moby Dick".
	4. In the context of "Moby Dick", how can we understand the literary treatment of human's fight against nature?
	5. How does Melville use the story of "Moby Dick" to illustrate and critique humanity's attempt to dominate nature?
-------------
How does Moby Dick explore the theme of obsession through Ahab's quest?
	1. In what ways does Herman Melville's "Moby Dick" depict the theme of relentless pursuit, as exemplified by Captain Ahab's fixation on the white whale?
	2. How does Melville use the character of Ahab in "Moby Dick" to delve into the psychological implications of obsession throughout the narrative?
	3. Explore how the

In [ ]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve

retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

<ipython-input-10-73df1d0ee15a>:10: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


2

## Generation

In [ ]:
from operator import itemgetter
from langchain_mistralai import ChatMistralAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatMistralAI(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

# final_rag_chain.invoke({"question":questions[1]})

for question in questions:
    print("-------------")
    print(question)
    rephrased_questions = generate_queries.invoke({"question":question})
    for rephrased_question in rephrased_questions:
        print("\t"+rephrased_question)
    print("\nAnswer: ")
    print(final_rag_chain.invoke({"question":question}))

-------------
What does Moby Dick say about humanity's struggle against nature?
	1. In what ways does Herman Melville's "Moby Dick" explore the theme of human conflict with nature?
	2. How does the novel "Moby Dick" portray the struggle between mankind and natural forces?
	3. Examine the depiction of humanity's battle against nature as presented in "Moby Dick".
	4. In the context of "Moby Dick", how can we understand the literary treatment of human's fight against nature?
	5. How does Melville use the story of "Moby Dick" to illustrate and critique humanity's attempt to dominate nature?

Answer: 
The novel "Moby Dick; Or, The Whale" by Herman Melville presents a complex exploration of humanity's struggle against nature, as represented by the white whale, Moby Dick. The protagonist, Captain Ahab, is driven by an obsession to exact revenge on the whale for previous encounters that left him scarred and determined. Ahab's fixation on Moby Dick symbolizes the human desire to conquer and dom